<!-- dom:TITLE: 1, 2, 3D models and linear systems -->
# 1, 2, 3D models and linear systems
**Aksel Hiorth**
University of Stavanger

Date: **May 6, 2024**

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

Most problems in nature are nonlinear. That means that the system response is not proportional to the system variables, e.g. doubling the CO$_2$ concentration in the atmosphere does not lead to a doubling of the earth surface temperature. Still, linear solvers lies at the heart of all grid based models describing e.g. the earths climate. The reason is that although the *global* model is nonlinear, the model can be formulated *locally* as a linear model. Typically the simulation code solves the nonlinear problem through a series of steps where each step is a solution of a linear problem. The topic of solving linear systems of equations have been extensively studied, and sophisticated linear equation solving packages have been developed. Python uses functions from the [LAPACK](https://en.wikipedia.org/wiki/LAPACK) library.

In the next sections we will show in detail how differential equations can be solved as a linear problem. We will first start off by deriving one of the most useful differential equations describing conservation of a quantity, e.g. mass, energy, momentum, charge.

# The continuity equation
The continuity equation is fundamental to all mathematical models describing a physical phenomenon. To gain more understanding of its origin we will take the time to derive it from first principles. We will do so in one dimension, consider a volume in space between $A(x)$ and $A(x+dx)$ in [figure 1](#fig:lin:flux). To be concrete we will assume that the green arrows represents the flow of heat. Thus there are heat flowing into and out of the system, and also heat that can be generated within the system by e.g. chemical reactions. The conservation equation can be formulated with words

<!-- dom:FIGURE: [fig-lin/flux.png, width=800 frac=0.5] A closed volume, $V(x)=A(x)dx$, where a quantity flows in and out (illustrated by the green lines), there is also a possibility for generation or loss of the same quantity inside the volume. <div id="fig:lin:flux"></div> -->
<!-- begin figure -->
<div id="fig:lin:flux"></div>

<img src="fig-lin/flux.png" width=800><p style="font-size: 0.9em"><i>Figure 1: A closed volume, $V(x)=A(x)dx$, where a quantity flows in and out (illustrated by the green lines), there is also a possibility for generation or loss of the same quantity inside the volume.</i></p>
<!-- end figure -->

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}
\frac{\text{heat into V(x)}}{\text{time}}-\frac{\text{heat out of V(x)}}{\text{time}}
+\frac{\text{heat generated in V(x)}}{\text{time}} {\nonumber}
\label{_auto1} \tag{1}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:flux"></div>

$$
\begin{equation}  
= \frac{\text{change of heat in V(x)}}{\text{time}}.
\label{eq:lin:flux} \tag{2}
\end{equation}
$$

We formulate the conservation equation per time, because we would like to investigate the time dependency of heat flow. The next step is to replace the terms ''heat into/out of'' with a useful mathematical quantity. It turns out that the term *flux* is particularly useful, because it is an *intensive* quantity. An intensive quantity is a quantity that is *independent of the system size*, like density. The flux is denoted by the symbol $J$

$$
\begin{equation}
J(x)=\frac{\text{quantity (heat)}}{\text{area}\cdot\text{time}},
\label{}
\end{equation}
$$

and was first introduced by Isaac Newton. Thus to find the amount of heat transported through a surface per time we simply multiply the flux with the surface area. Next, we define the heat per volume as $q(x)$, and the heat produced per volume as $\sigma$. Then equation ([2](#eq:lin:flux)) can be written

<!-- Equation labels as ordinary links -->
<div id="_auto2"></div>

$$
\begin{equation}
\frac{J(x)A(x)}{dt}-\frac{J(x+dx)A(x+dx)}{dt}+
\frac{\sigma(t+dt)V(x)-\sigma(t)V(x)}{dt}{\nonumber}
\label{_auto2} \tag{3}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:cont1"></div>

$$
\begin{equation} =
\frac{q(t+dt)V(x)-q(t)V(x)}{dt}.
\label{eq:lin:cont1} \tag{4}
\end{equation}
$$

Using Taylor expansion we can write

<!-- Equation labels as ordinary links -->
<div id="eq:lin:cont2"></div>

$$
\begin{equation}
J(x+dx)A(x+dx) =J(x)A(x)+\frac{d(J(x)A(x)}{dx}dx+{\cal O}(dx^2),
\label{eq:lin:cont2} \tag{5} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto3"></div>

$$
\begin{equation}  
\sigma(t+dt) =\sigma(t)+\frac{d\sigma}{dt}dt+{\cal O}(dt^2),{\nonumber}
\label{_auto3} \tag{6}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:cont3"></div>

$$
\begin{equation}  
q(t+dt) =q(t)+\frac{dq}{dt}dt+{\cal O}(dt^2),
\label{eq:lin:cont3} \tag{7}
\end{equation}
$$

Inserting these equations into equation([4](#eq:lin:cont1)), using $V(x)=A(x)dx$, and taking the limit $dx,dt\to0$ we arrive at
**The continuity equation in 1 dimension.**

<!-- Equation labels as ordinary links -->
<div id="eq:lin:cont4"></div>

$$
\begin{equation}
-\frac{d(J(x)A(x))}{dx}+\frac{d\sigma(t)}{dt}A(x)=\frac{dq(t)}{dt}A(x).
\label{eq:lin:cont4} \tag{8}
\end{equation}
$$

We have kept the area in equation ([8](#eq:lin:cont4)), because we are only considering flow of heat in one dimension and then we can allow for the area to change in the $y$ and $z$ dimension. When the continuity equation is derived in three dimensions, one consider a volume $V(x,y,z)=dxdydz$, then the area in equation ([8](#eq:lin:cont4)) will drop out and $d/dx\to\nabla=[\partial/\partial x, \partial/\partial y, \partial/\partial z]$ 
**The continuity equation in 3 dimensions.**

<!-- Equation labels as ordinary links -->
<div id="eq:lin:cont5"></div>

$$
\begin{equation}
-\nabla\cdot\mathbf{J}+\frac{d\sigma(t)}{dt}=\frac{dq(t)}{dt}.
\label{eq:lin:cont5} \tag{9}
\end{equation}
$$

# Continuity equation as a linear problem

How can a differential equation be formulated as a matrix problem? To see this we need to discretize equation ([8](#eq:lin:cont4)). We will discretize the equation in one dimension, and we will use a regular grid, where we keep the same distance, $h$, between the points. Assume our system has dimension $L$, in [figure 2](#fig:lin:grid), there are two examples of discretization.  

<!-- dom:FIGURE: [fig-lin/grid.png, width=800 frac=1.0] Examples of discretization of a system with length $L$ (left) the  boundaries lies exactly at the boundary nodes, (right) boundary nodes lies half-way between the grid nodes. <div id="fig:lin:grid"></div> -->
<!-- begin figure -->
<div id="fig:lin:grid"></div>

<img src="fig-lin/grid.png" width=800><p style="font-size: 0.9em"><i>Figure 2: Examples of discretization of a system with length $L$ (left) the  boundaries lies exactly at the boundary nodes, (right) boundary nodes lies half-way between the grid nodes.</i></p>
<!-- end figure -->

There are many things to consider when discretizing equations, but perhaps the most important are
1. Treat the boundary nodes correctly. In most cases the dominating numerical errors are introduced through the boundaries. Always draw a picture of the system, if the boundaries lies exactly at the grid nodes it is usually easier to find a good numerical representation. If the boundaries lies a distance from the nodes, e.g. to the right in [figure 2](#fig:lin:grid), then one usually need to do some interpolation.

2. Should finite volume or finite difference approach be used? A finite volume approach is especially attractive for conservation equations.

**Finite difference and finite volume.**

We have already encountered finite difference discretization in the last chapter where we used various approximations to calculate derivatives, i.e. we calculate derivatives by calculating the *difference* between $f(x+h)$ and $f(x)$ (or $f(x+h)$ and $f(x-h)$). The finite volume formulation is also a finite difference scheme, but it is formulated such that we always ensure that the quantity we are simulating is conserved (regardless of numerical errors). Formally, one transforms the divergence term (the term that contains the flux $\nabla \mathbf{J}$) into a surface integral using the Gauss (divergence) theorem

<!-- Equation labels as ordinary links -->
<div id="eq:lin:dgau"></div>

$$
\begin{equation}
\int_V\nabla\cdot \mathbf{J}=\int_S\mathbf{J}\cdot \hat{\mathbf{n}}
\label{eq:lin:dgau} \tag{10}
\end{equation}
$$

There are excellent books written on the finite volume method, see e.g. [[leveque2002finite]](#leveque2002finite). Here we will mainly focus on the key idea, which is to formulate a scheme that conserves the flux.
The process of formulating a finite volume scheme is very close to the derivation of the continuum equation we did in the beginning of the chapter. We consider our numerical discretization as several boxes (exactly like the dotted lines in [figure 2](#fig:lin:grid)), the continuum equation is written down for each box an therefore we are ensured that the quantities are conserved *regardless of the size of the boxes*.

**Example: Finite difference and volume discretization of the heat equation.**

Let us consider the heat equation, where the heat flux is given as

<!-- Equation labels as ordinary links -->
<div id="eq:lin:e1"></div>

$$
\begin{equation}
J=-k\frac{dT}{dx},
\label{eq:lin:e1} \tag{11}
\end{equation}
$$

where $k$ describes the thermal conductivity of the solid. We will further assume that there is a constant source term $d\sigma/dt=\kappa=const$, and steady state $dq/dt=0$. Then equation ([8](#eq:lin:cont4)) can be written

<!-- Equation labels as ordinary links -->
<div id="eq:lin:e2"></div>

$$
\begin{equation}
k\frac{d^2T}{dx^2}+\kappa=0,
\label{eq:lin:e2} \tag{12}
\end{equation}
$$

The finite difference discretization is now straight forward, just replace the term $d^2T/dx^2$ with a suitable finite difference formula for the second derivative, e.g.

<!-- Equation labels as ordinary links -->
<div id="_auto4"></div>

$$
\begin{equation}
k\frac{T(x+h)+T(x-h)-2T(x)}{h^2}+\kappa=0,{\nonumber}
\label{_auto4} \tag{13}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:e3"></div>

$$
\begin{equation}  
k\frac{T_{i+1}+T_{i-1}-2T_i}{h^2}+\kappa=0.
\label{eq:lin:e3} \tag{14}
\end{equation}
$$

Note that in the last equations we have introduced the short hand notation $T(x)\equiv T_i$, and $T(x\pm h)=T_{i\pm 1}$. 

The finite volume discretization approach is slightly different, we then operate with *cell averaged values*. The heat in the box is the volume averaged heat. Since the divergence term is replaced with a surface integral, equation ([10](#eq:lin:dgau)), we calculate the flow of heat into the boundary $x-h/2$ and out of the boundary $x+h/2$ as

<!-- Equation labels as ordinary links -->
<div id="eq:lin:e4"></div>

$$
\begin{equation}
\frac{J_{x+h/2}
-J_{x-h/2}}{h}+\kappa=0.
\label{eq:lin:e4} \tag{15}
\end{equation}
$$

Note that this equation is exactly the same as equation ([4](#eq:lin:cont1)), with the only exception that the point $x$ is placed in the center of the box.
The diffusive flux is $-kdT/dx$, and in order to be consistent with this law we have to write the flux between two cells as proportional to the difference between the cell average values

$$
-\frac{k}{h}\left(\frac{T_{i+1}-T_i}{h}-\frac{T_{i}-T_{i-1}}{h}\right)+\kappa=0\nonumber,
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:e5"></div>

$$
\begin{equation}  
k\frac{T_{i+1}+T_{i-1}-2T_i}{h^2}+\kappa=0.
\label{eq:lin:e5} \tag{16}
\end{equation}
$$

In this case we actually recover the same equation as we did for the finite difference approach equation ([14](#eq:lin:e3)).



### Boundary conditions

Basically there are two types of boundary conditions i) the flux is known at the edges of the computational domain and/or ii) the physical quantity we are solving for is known. To be more specific, and to see how all connects, we will continue with the example above on the heat equation. Consider the outline of nodes as in [figure 2](#fig:lin:grid), we will consider two possibilities i) where the physical boundary lies exactly between nodes, and ii) where the physical boundary is exactly at the grid nodes. In the finite volume scheme, we need to make sure that the flux over the surface is calculated correctly, and then we have to use the formulas in [figure 3](#fig:lin:bbc)

<!-- Equation labels as ordinary links -->
<div id="eq:lin:nn"></div>

$$
\begin{equation}
\left.\frac{dT}{dx}\right|_{x=0}=\frac{T_{-1}-T_0}{h}+{\cal O}(h^2).
\label{eq:lin:nn} \tag{17}
\end{equation}
$$

Note that if the boundary node lies exactly at $x=0$, we have to replace $T_0$ with $T_{1}$. A flux boundary condition is usually called Neumann boundary condition after Carl Neumann (1832–1925) a German mathematician, and the constant value boundary condition is called Dirichlet boundary condition after another German mathematician, Peter Gustav Lejeune Dirichlet (1805–1859). If the boundary nodes lies exactly at the physical boundary, it is trivial to implement, just replace $T_N=T_b$ i.e. with the boundary value. On the other hand if the physical boundary lies a distance from the node, we have to interpolate the value from the physical coordinate to the simulation node.

<!-- Equation labels as ordinary links -->
<div id="_auto5"></div>

$$
\begin{equation}
T_N=T(x+h)=T(x+h/2+h/2){\nonumber}
\label{_auto5} \tag{18}
\end{equation}
$$

$$
=T_{N+1/2}+\left.\frac{dT}{dx}\right|_{x+h/2}+{\cal O}(h^2)=T_b+\frac{T_N-T_{N-1}}{h}\frac{h}{2}+{\cal O}(h^2),\text{ hence:}\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:dd"></div>

$$
\begin{equation}  
T_N=2T_b-T_{N-1}+{\cal O}(h^2).\label{eq:lin:dd} \tag{19}
\end{equation}
$$

Notice that the result make sense, $T_b=(T_N+T_{N-1})/2$, i.e. the value midway is the average of the values at the neighboring nodes.

<!-- dom:FIGURE: [fig-lin/bbc.png, width=800 frac=1.0] Flux boundary condition (Neumann), and value boundary condition (Dirichlet). For the upper right boundary condition we use Taylors formula to interpolate, see equation ([19](#eq:lin:dd)). <div id="fig:lin:bbc"></div> -->
<!-- begin figure -->
<div id="fig:lin:bbc"></div>

<img src="fig-lin/bbc.png" width=800><p style="font-size: 0.9em"><i>Figure 3: Flux boundary condition (Neumann), and value boundary condition (Dirichlet). For the upper right boundary condition we use Taylors formula to interpolate, see equation ([19](#eq:lin:dd)).</i></p>
<!-- end figure -->

**Example: Steady state heat equation as a linear problem.**

Consider the case where we have 4 grid nodes and the outline of the simulation nodes are as in [figure 2](#fig:lin:grid) to the left, i.e. nodes at the physical  boundaries. Assume a zero flux boundary condition to the left, and a constant temperature,$T_b$, to the right. Write the heat equation

<!-- Equation labels as ordinary links -->
<div id="eq:lin:exx1"></div>

$$
\begin{equation}
k\frac{d^2T}{dx^2}+\kappa=0,
\label{eq:lin:exx1} \tag{20}
\end{equation}
$$

as a matrix equation.

**Solution:**
First, we use the discrete version of equation ([20](#eq:lin:exx1)) in equation ([14](#eq:lin:e3)) for $i=0, 1, 2, 3$

$$
T_{-1}+T_1-2T_0 =-h^2\kappa/k,\nonumber
$$

$$
T_{0}+T_2-2T_1 =-h^2\kappa/k\nonumber
$$

$$
T_{1}+T_3-2T_2 =-h^2\kappa/k\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:exx2"></div>

$$
\begin{equation}  
T_{2}+T_4-2T_3 =-h^2\kappa/k.
\label{eq:lin:exx2} \tag{21}
\end{equation}
$$

Now, we have four equations, but six unknowns ($T_{-1}, T_0, T_1, T_2, T_3, T_4$). $T_{-1}$, and $T_4$ can be found from the boundary conditions. Using the formulas in [figure 3](#fig:lin:bbc) at the lower left and lower right, we get $dT/dx=0$, and $T_{-1}=T_1$, and $T_4=T_b$. Thus the first and last equation in equation ([21](#eq:lin:exx2)), can be written

$$
2T_1-2T_0 =-h^2\kappa/k,\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:2b"></div>

$$
\begin{equation}  
T_{2}-2T_3 =-h^2\kappa/k-T_b.
\label{eq:lin:2b} \tag{22}
\end{equation}
$$

Now, we can formulate equation ([21](#eq:lin:exx2)) as a matrix problem, with the unknowns on the left side and the unknown on the right hand side.

<!-- Equation labels as ordinary links -->
<div id="eq:lin:exx4"></div>

$$
\begin{equation}
\left(
\begin{array}{cccc}
-2&2&0&0\\ 
1&-2&1&0\\ 
0&1&-2&1\\ 
0&0&1&-2\\ 
\end{array}
\right)
\left(
\begin{array}{c}
T_0\\ 
T_1\\ 
T_2\\ 
T_3\\ 
\end{array}
\right)
=
\left(
\begin{array}{c}
-h^2\kappa/k\\ 
-h^2\kappa/k\\ 
-h^2\kappa/k\\ 
-h^2\kappa/k-T_b
\end{array}
\right).
\end{equation}
\label{eq:lin:exx4} \tag{23}
$$

In principle, to discretize an equation is straight forward, but there are some 

First, we are going to consider a *steady state* solution. Steady state means that the solution does no longer change as a function of time, i.e. $dq/dt=0$ in equation ([8](#eq:lin:cont4)). We are also going to assume that the area is constant $A(x)=A$, thus the equation we want to solve is

# Solving linear equations
There are a number of excellent books covering this topic, see e.g. [[press2007;@trefethen1997;@stoer2013;@strang2019]](#press2007;@trefethen1997;@stoer2013;@strang2019).
In most of the examples covered in this course we will encounter problems where we have a set of *linearly independent* equations and one equation for each unknown. For these type of problems there are a number of methods that can be used, and they will find a solution in a finite number of steps. If a solution cannot be found it is usually because the equations are not linearly independent, and our formulation of the physical problem is wrong.

Assume that we would like to solve the following set of equations:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:la"></div>

$$
\begin{equation}
2x_0+x_1+x_2+3x_3=1, \label{eq:lin:la} \tag{24} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:lb"></div>

$$
\begin{equation}  
x_0+x_1+3x_2+x_3=-3, \label{eq:lin:lb} \tag{25} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:lc"></div>

$$
\begin{equation}  
x_0+4x_1+x_2+x_3=2, \label{eq:lin:lc} \tag{26} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:ld"></div>

$$
\begin{equation}  
x_0+x_1+2x_2+2x_3=1. \label{eq:lin:ld} \tag{27} 
\end{equation}
$$

These equations can be written in matrix form as:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:mat"></div>

$$
\begin{equation}
\mathbf{A\cdot x}=\mathbf{b},
\label{eq:lin:mat} \tag{28}
\end{equation}
$$

where:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:matA"></div>

$$
\begin{equation}
\mathbf{A}\equiv\begin{pmatrix}
2&1&1&3\\ 
1&1&3&1\\ 
1&4&1&1\\ 
1&1&2&2
\end{pmatrix}
\qquad
\mathbf{b}\equiv
\begin{pmatrix}
1\\-3\\2\\1
\end{pmatrix}
\qquad
\mathbf{x}\equiv
\begin{pmatrix}
x_0\\x_1\\x_2\\x_3
\end{pmatrix}.
\label{eq:lin:matA} \tag{29}
\end{equation}
$$

You can easily verify that $x_0=-4, x_1=1, x_2=-1, x_3= 3$ is the
solution to the above equations by direct substitution. If we were to
replace one of the above equations with a linear combination of any of
the other equations, e.g. replace equation ([27](#eq:lin:ld)) with
$3x_0+2x_1+4x_2+4x_3=-2$, there would be no unique solution (infinite
number of solutions). This can be checked by calculating the determinant of the matrix $\mathbf{A}$, if $\det \mathbf{A}=0 $,  
What is the difficulty in solving these equations? Clearly if none of the equations are linearly dependent, and we have $N$ independent linear equations, it should be straight forward to solve them? Two major numerical problems are i) even if the equations are not exact linear combinations of each other, they could be very close, and as the numerical algorithm progresses they could at some stage become linearly dependent due to roundoff errors. ii) roundoff errors may accumulate if the number of equations are large [[press2007]](#press2007).

## Gauss-Jordan elimination
Let us continue the discussion by consider Gauss-Jordan elimination, which is a *direct* method. A direct method uses a final set of operations to obtain a solution. According to [[press2007]](#press2007) Gauss-Jordan elimination is the method of choice if we want to find the inverse of $\mathbf{A}$. However, it is slow when it comes to calculate the solution of equation
([28](#eq:lin:mat)). Even if speed and memory use is not an issue, it is also not advised to first find the inverse, $\mathbf{A}^{-1}$, of $\mathbf{A}$, then multiply it with $\mathbf{b}$ to obtain the solution, due to roundoff errors (Roundoff errors occur whenever we subtract to numbers that are very close to each other). To simplify our notation, we write equation ([29](#eq:lin:matA)) as:

<!-- Equation labels as ordinary links -->
<div id="_auto6"></div>

$$
\begin{equation}
\left(
\begin{array}{cccc|c}
2&1&1&3&1\\ 
1&1&3&1&-3\\ 
1&4&1&1&2\\ 
1&1&2&2&1
\end{array}
\right).
\label{_auto6} \tag{30}
\end{equation}
$$

The numbers to the left of the vertical dash is the matrix $\mathbf{A}$, and to the right is the vector $\mathbf{b}$. The Gauss-Jordan elimination procedure proceeds by doing the same operation on the right and left side of the dash, and the goal is to get only zeros on the lower triangular part of the matrix. This is achieved by multiplying rows with the same (nonzero) number, swapping rows, adding a multiple of a row to another:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:gj1"></div>

$$
\begin{equation}
\left(
\begin{array}{cccc|c}
2&1&1&3&1\\ 
1&1&3&1&-3\\ 
1&4&1&1&2\\ 
1&1&2&2&1
\end{array}
\right)\to
\left(
\begin{array}{cccc|c}
2&1&1&3&1\\ 
0&1/2&5/2&-1/2&-7/2\\ 
0&7/2&1/2&-1/2&3/2\\ 
0&1/2&3/2&1/2&1/2
\end{array}
\right)\to\label{eq:lin:gj1} \tag{31}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto7"></div>

$$
\begin{equation}  
\left(
\begin{array}{cccc|c}
2&1&1&3&1\\ 
0&1/2&5/2&-1/2&-7/2\\ 
0&0&-17&3&26\\ 
0&0&1&-1&4
\end{array}
\right)
\to
\left(
\begin{array}{cccc|c}
2&1&1&3&1\\ 
0&1/2&5/2&-1/2&-7/2\\ 
0&0&-17&3&26\\ 
0&0&0&14/17&42/17
\end{array}
\right){\nonumber}
\label{_auto7} \tag{32}
\end{equation}
$$

The operations done are: ($1\to2$) multiply first row with $-1/2$ and add to second, third and the fourth row, ($2\to 3$) multiply second row with $-7$, and add to third row, multiply second row with $-1$ and add to fourth row, ($3\to4$) multiply third row with $-1/17$ and add to fourth row. These operations can easily be coded into Python:

In [2]:
A = np.array([[2, 1, 1, 3],[1, 1, 3, 1],
              [1, 4, 1, 1],[1, 1, 2, 2 ]],float)
b = np.array([1,-3,2,1],float)
N=4
# Gauss-Jordan Forward Elimination
for i in range(1,N):
    fact    = A[i:,i-1]/A[i-1,i-1]
    A[i:,] -= np.outer(fact,A[i-1,])
    b[i:]  -= b[i-1]*fact

The python code is a bit compact, below there is an implementation using for loops

In [3]:
# Gauss-Jordan Forward Elimination - for loops
for i in range(N):
    for j in range(i+1,N):
        fact    = A[j,i]/A[i,i]
        for k in range(i+1,N):
            A[j,k] = A[j,k]- fact*A[i,k]
        b[j]  = b[j]- b[j-1]*fact
	A[j,i]= 0. # alternatively k=i,...,N

**Number of (long) operations.**

The code above reveals that that there are quite a few multiplications or divisions being performed in the forward elimination. Multiplications and divisions are more time consuming than addition and subtraction, and are usually termed *long* operations. Not all loops runs from zero to $N$, the innermost from $k=i+1\ldots N-1$, i.e. a total of $N-i-2$, the second contains $N-i-2$ and one multiplication for the `b` vector. Hence we have number of long operations

$$
\begin{equation}
\sum_{i=0}^{N-1}(N-i-2)^2+(N-i-2)=\frac{N}{3}(N^2-3N+2).
\label{}
\end{equation}
$$

The important result is that when the system of equations becomes large $N^3\gg N^2$ and the algorithm scales as $N^3$.


Notice that the final matrix has only zeros beyond the diagonal, such a matrix is called *upper triangular*. We still have not found the final solution, but from an upper triangular (or lower triangular) matrix it is trivial to determine the solution. The last row immediately gives us $14/17z=42/17$ or $z=3$, now we have the solution for z and the next row gives: $-17y+3z=26$ or $y=(26-3\cdot3)/(-17)=-1$, and so on. In a more general form, we can write our solution of the matrix $\mathbf{A}$ after making it upper triangular as:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:back"></div>

$$
\begin{equation}
\begin{pmatrix}
a^\prime_{0,0}&a^\prime_{0,1}&a^\prime_{0,2}&a^\prime_{0,3}\\ 
0&a^\prime_{1,1}&a^\prime_{1,2}&a^\prime_{1,3}\\ 
0&0&a^\prime_{2,2}&a^\prime_{2,3}\\ 
0&0&0&a^\prime_{3,3}
\end{pmatrix}
\cdot
\begin{pmatrix}
x_0\\ 
x_1\\ 
x_2\\ 
x_3
\end{pmatrix}
=
\begin{pmatrix}
b^\prime_{0}\\ 
b^\prime_{1}\\ 
b^\prime_{2}\\ 
b^\prime_{3}
\end{pmatrix}
\label{eq:lin:back} \tag{33}
\end{equation}
$$

The back substitution can then be written formally as:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:back2"></div>

$$
\begin{equation}
x_i=\frac{1}{a^\prime_{ii}}\left[b_i^\prime-\sum_{j=i+1}^{N-1}a^\prime_{ij}x_j\right],\quad i=N-1,N-2,\ldots,0
\label{eq:lin:back2} \tag{34}
\end{equation}
$$

The back substitution can now easily be implemented in Python as:

In [4]:
# Back substitution
sol = np.zeros(N,float)
sol[N-1]=b[N-1]/A[N-1,N-1]
for i in range(2,N+1):
    sol[N-i]=(b[N-i]-np.dot(A[(N-i),],sol))/A[N-i,N-i]

Notice that in the Python implementation, we have used vector operations instead of for loops. This makes the code more efficient, but it could also be implemented with for loops:

In [5]:
# Back substitution - for loop
sol = np.zeros(N,float)
for i in range(N-1,-1,-1):
    sol[i]= b[i]
    for j in range(i+1,N):
        sol[i] -= A[i][j]*sol[j]
    sol[i] /= A[i][i]

**Number of (long) operations.**

As for the forward elimination, we can find how the backward substitution scales. Notice that here there are only two loops, hence we have number of long operations

$$
\begin{equation}
\sum_{i=0}^{N-1}(N-i-1)=\frac{N}{2}(N-1).
\label{}
\end{equation}
$$

Thus, the backward substitution scales as $N^2$.


There are at least two things to notice with our implementation:
* Matrix and vector notation makes the code more compact and efficient. In order to understand the implementation it is advised to put $i=1, 2, 3, 4$, and then execute the statements in the Gauss-Jordan elimination and compare with equation ([31](#eq:lin:gj1)).

* The implementation of the Gauss-Jordan elimination is not robust, in particular one could easily imagine cases where one of the leading coefficients turned out as zero, and the routine would fail when we divide by `A[i-1,i-1]`. By simply changing equation ([25](#eq:lin:lb)) to $2x_0+x_1+3x_2+x_3=-3$, when doing the first Gauss-Jordan elimination, both $x_0$ and $x_1$ would be canceled. In the next iteration we try to divide next equation by the leading coefficient of $x_1$, which is zero, and the whole procedure fails.

## Pivoting
The solution to the last problem is solved by what is called *pivoting*. The element that we divide on is called the *pivot element*. It actually turns out that even if we do Gauss-Jordan elimination *without* encountering a zero pivot element, the Gauss-Jordan procedure is numerically unstable in the presence of roundoff errors [[press2007]](#press2007). There are two versions of pivoting, *full pivoting* and *partial pivoting*. In partial pivoting we only interchange rows, while in full pivoting we also interchange rows and columns. Partial pivoting is much easier to implement, and the algorithm is as follows:
1. Find the row in $\mathbf{A}$ with largest absolute value in front of $x_0$ and change with the first equation, switch corresponding elements in $\mathbf{b}$

2. Do one Gauss-Jordan elimination, find the row in $\mathbf{A}$ with the largest absolute value in front of $x_1$ and switch with the second (same for $\mathbf{b}$), and so on.

For a linear equation we can multiply with a number on each side and the equation would be unchanged, so if we where to multiply one of the equations with a large value, we are almost sure that this equation would be placed first by our algorithm. This seems a bit strange as our mathematical problem is the same. Sometimes the linear algebra routines tries to normalize the equations to find the pivot element that would have been the largest element if all equations were normalized according to some rule, this is called *implicit pivoting*.  
## LU decomposition
As we have already seen, if the matrix $\mathbf{A}$ is reduced to a triangular form it is trivial to calculate the solution by using back substitution. Thus if it was possible to decompose the matrix $\mathbf{A}$ as follows:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:lu"></div>

$$
\begin{equation}
\mathbf{A}=\mathbf{L}\cdot\mathbf{U}\label{eq:lin:lu} \tag{35}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto8"></div>

$$
\begin{equation}
\begin{pmatrix}
a_{0,0}&a_{0,1}&a_{0,2}&a_{0,3}\\ 
a_{1,0}&a_{1,1}&a_{1,2}&a_{1,3}\\ 
a_{2,0}&a_{2,1}&a_{2,2}&a_{2,3}\\ 
a_{3,0}&a_{3,1}&a_{3,2}&a_{3,3}
\end{pmatrix}
=
\begin{pmatrix}
l_{0,0}&0&0&0\\ 
l_{1,0}&l_{1,1}&0&0\\ 
l_{2,0}&l_{2,1}&l_{2,2}&0\\ 
l_{3,0}&l_{3,1}&l_{3,2}&l_{3,3}
\end{pmatrix}
\cdot
\begin{pmatrix}
u_{0,0}&u_{0,1}&u_{0,2}&u_{0,3}\\ 
0&u_{1,1}&u_{1,2}&u_{1,3}\\ 
0&0&u_{2,2}&u_{2,3}\\ 
0&0&0&u_{3,3}
\end{pmatrix}.
\label{_auto8} \tag{36}
\end{equation}
$$

The solution procedure would then be to rewrite equation ([28](#eq:lin:mat)) as:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:matb"></div>

$$
\begin{equation}
\mathbf{A\cdot x}=\mathbf{L}\cdot\mathbf{U}\cdot\mathbf{x}=\mathbf{b},\label{eq:lin:matb} \tag{37}
\end{equation}
$$

If we define a new vector $\mathbf{y}$:

<!-- Equation labels as ordinary links -->
<div id="_auto9"></div>

$$
\begin{equation}
\mathbf{y}\equiv\mathbf{U}\cdot\mathbf{x},
\label{_auto9} \tag{38}
\end{equation}
$$

we can first solve for the $\mathbf{y}$ vector:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:for"></div>

$$
\begin{equation}
\mathbf{L}\cdot\mathbf{y}=\mathbf{b},\label{eq:lin:for} \tag{39}
\end{equation}
$$

and then for $\mathbf{x}$:

<!-- Equation labels as ordinary links -->
<div id="_auto10"></div>

$$
\begin{equation}
\mathbf{U}\cdot\mathbf{x}=\mathbf{y}.
\label{_auto10} \tag{40}
\end{equation}
$$

Note that the solution to equation ([39](#eq:lin:for)) would be done by *forward substitution*:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:back3"></div>

$$
\begin{equation}
y_i=\frac{1}{l_{ii}}\left[b_i-\sum_{j=0}^{i-1}l_{ij}x_j\right],\quad i=1,2,\ldots N-1.
\label{eq:lin:back3} \tag{41}
\end{equation}
$$

Why go to all this trouble? First of all it requires (slightly) less operations to calculate the LU decomposition and doing the forward and backward substitution than the Gauss-Jordan procedure discussed earlier. Secondly, and more importantly, is the fact that in many cases one would like to calculate the solution for different values of the $\mathbf{b}$ vector in equation ([37](#eq:lin:matb)). If we do the LU decomposition first we can calculate the solution quite fast using backward and forward substitution for any value of the $\mathbf{b}$ vector.

The NumPy function [`solve`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html), uses LU decomposition and partial pivoting, and we can find the solution to our previous problem simply by the following code:

In [6]:
from numpy.linalg import solve
x=solve(A,b)

# Iterative methods
The methods described so far are what is called *direct* methods. The direct methods for very large systems might suffer from round off errors. That means that even if the computer has found a solution, the solution is "polluted" by round off errors, or stated more clearly: your solution for $\mathbf{x}$, when entered into the original equation $\mathbf{A}\mathbf{x}\neq\mathbf{b}$. Below we will describe one trick, and two alternative methods to the direct methods.
## Iterative improvement
The first method [[press2001]](#press2001) assumes that we already have solved the matrix equation ([28](#eq:lin:mat)), and obtained an *estimate* $\mathbf{\hat{x}}$ of the true solution $\mathbf{x}$. Assume that $\mathbf{\hat{x}}=\mathbf{x}+\delta\mathbf{x}$, and that

<!-- Equation labels as ordinary links -->
<div id="eq:lin:itb"></div>

$$
\begin{equation}
\mathbf{A}\cdot\mathbf{\hat{x}}=\mathbf{A}\cdot(\mathbf{x}+\delta\mathbf{x})=\mathbf{b}+\delta\mathbf{b},
\label{eq:lin:itb} \tag{42}
\end{equation}
$$

subtracting equation ([28](#eq:lin:mat)) we get

<!-- Equation labels as ordinary links -->
<div id="eq:lin:itb2"></div>

$$
\begin{equation}
\mathbf{A}\cdot\delta\mathbf{x}=\delta\mathbf{b}.
\label{eq:lin:itb2} \tag{43}
\end{equation}
$$

Solving equation ([42](#eq:lin:itb)) for $\delta\mathbf{b}$ an inserting in the equation above, we get

<!-- Equation labels as ordinary links -->
<div id="eq:lin:itb3"></div>

$$
\begin{equation}
\mathbf{A}\cdot\delta\mathbf{x}=\mathbf{A}\cdot\mathbf{\hat{x}}-\mathbf{b}.
\label{eq:lin:itb3} \tag{44}
\end{equation}
$$

The usefulness of this method assumes that we have already obtained the LU decomposition of $\mathbf{A}$, and if possible one should use a higher precision to calculate the right hand side, since there will be a lot of cancellations. Then the whole computational process it is simply to calculate the right hand side and backsubstitute. The improved solution is then obtained by subtracting $\delta\mathbf{x}$ from $\mathbf{\hat{x}}$.

## The Jacobi method
A completely different approach is the Jacobian method, which is simply to decompose the $\mathbf{A}$ matrix in the following way

<!-- Equation labels as ordinary links -->
<div id="eq:lin:DR"></div>

$$
\begin{equation}
\mathbf{A}=\mathbf{D}+\mathbf{R}
\label{eq:lin:DR} \tag{45}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto11"></div>

$$
\begin{equation}
&\begin{pmatrix}
a_{0,0}&a_{0,1}&a_{0,2}&a_{0,3}\\ 
a_{1,0}&a_{1,1}&a_{1,2}&a_{1,3}\\ 
a_{2,0}&a_{2,1}&a_{2,2}&a_{2,3}\\ 
a_{3,0}&a_{3,1}&a_{3,2}&a_{3,3}
\end{pmatrix}
{\nonumber}
\label{_auto11} \tag{46}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:DR2"></div>

$$
\begin{equation}  
=
\begin{pmatrix}
a_{0,0}&0&0&0\\ 
0&a_{1,1}&0&0\\ 
0&0&a_{2,2}&0\\ 
0&0&0&a_{3,3}
\end{pmatrix}
+
\begin{pmatrix}
0&a_{0,1}&a_{0,2}&a_{0,3}\\ 
a_{1,0}&0&a_{1,2}&a_{1,3}\\ 
a_{2,0}&a_{2,1}&0&a_{2,3}\\ 
a_{3,0}&a_{3,1}&a_{3,2}&0
\end{pmatrix}.
\label{eq:lin:DR2} \tag{47}
\end{equation}
$$

We can then write equation ([28](#eq:lin:mat)) as

<!-- Equation labels as ordinary links -->
<div id="eq:lin:jc"></div>

$$
\begin{equation}
\mathbf{D}\mathbf{x}=\mathbf{b}-\mathbf{R}\cdot\mathbf{x}.
\label{eq:lin:jc} \tag{48}
\end{equation}
$$

How does this help us? First of all, the matrix $\mathbf{D}$ is easy to invert as it is diagonal, the inverse can be found by simply replace $a_{ii}\to 1/a_{ii}$. But $\mathbf{x}$ is still present on the right hand side? This is where the *iterations* comes into play, we simply guess at an initial solution $\mathbf{x}^k$, and then we use equation ([48](#eq:lin:jc)) to calculate the next solution $\mathbf{x}^{k+1}$, and so on

<!-- Equation labels as ordinary links -->
<div id="eq:lin:jc2"></div>

$$
\begin{equation}
\mathbf{x}^{k+1}=\mathbf{D}^{-1}(\mathbf{b}-\mathbf{R}\cdot\mathbf{x}^{k}).
\label{eq:lin:jc2} \tag{49}
\end{equation}
$$

Lets write it out on component form for a $4\times4$ matrix to see what is going on

<!-- Equation labels as ordinary links -->
<div id="eq:lin:jc3a"></div>

$$
\begin{equation}
x_0^{k+1} =\frac{1}{a_{00}}(b_0-a_{01}x_1^k-a_{02}x_2^k-a_{03}x_3^k),
\label{eq:lin:jc3a} \tag{50}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:jc3b"></div>

$$
\begin{equation}  
x_1^{k+1} =\frac{1}{a_{11}}(b_1-a_{10}x_0^k-a_{12}x_2^k-a_{13}x_3^k),
\label{eq:lin:jc3b} \tag{51}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:jc3c"></div>

$$
\begin{equation}  
x_2^{k+1} =\frac{1}{a_{22}}(b_2-a_{20}x_0^k-a_{21}x_1^k-a_{23}x_3^k),
\label{eq:lin:jc3c} \tag{52}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:jc3d"></div>

$$
\begin{equation}  
x_3^{k+1} =\frac{1}{a_{33}}(b_3-a_{30}x_0^k-a_{31}x_1^k-a_{32}x_2^k).
\label{eq:lin:jc3d} \tag{53}
\end{equation}
$$

Below is a Python implementation

In [7]:
def solve_jacobi(A,b,x=-1,w=1,max_iter=1000,EPS=1e-6):
    """
    Solves the linear system Ax=b using the Jacobian method, stops if
    solution is not found after max_iter or if solution changes less 
    than EPS
    """
    if(x==-1): #default guess 
        x=np.zeros(len(b))
    D=np.diag(A)
    R=A-np.diag(D)
    eps=1
    x_old=x
    iter=0
    w=0.1
    while(eps>EPS and iter<max_iter):
        iter+=1
        x=w*(b-np.dot(R,x_old))/D + (1-w)*x_old
        eps=np.sum(np.abs(x-x_old))
        x_old=x
    print('found solution after ' + str(iter) +' iterations')
    return x

A sufficient criteria for the Jacobian method to converge is if the matrix $A$ is diagonally dominant. In the implementation above we have included a weight, which sometimes can help in the convergence even if the matrix is not diagonally dominant. 
Test out the following examples, note that by rearranging the problem we can achieve convergence.

In [8]:
A = np.array([[10, -1, 2,0],[-1, 11, -1,3],
               [2, -1, 10, -1],[0, 3, -1, 8 ]],float)
b = np.array([6,25,-11,15],float)
print(A)
s1=solve_jacobi(A,b)
print(s1)

A = np.array([[2, 1, 1, 3],[1, 1, 3, 1],
              [1, 4, 1, 1],[1, 1, 2, 2 ]],float)
b = np.array([1,-3,2,1],float)
#try w=1, and w=0.01
s2=solve_jacobi(A,b,w=1)
print(s2)

#exchange row 3 and 4, and put w=0.1
A = np.array([[2, 1, 1, 3,],[1, 4, 1, 1, ],
              [1, 1, 3, 1],[1, 1, 2, 2 ]],float)
b = np.array([1,2,-3,1],float)
s3=solve_jacobi(A,b,w=0.1)
print(s3)

The iterative method can be appealing if we do not need a high accuracy, we can choose to stop whenever $|\mathbf{x}^{k+1}-\mathbf{x}^k|$ is small enough. For the direct method we have to follow through all the way.
**Convergence.**

The Jacobi method converges if the matrix $\mathbf{A}$ is strictly diagonally dominant. Strictly diagonally dominant means that the absolute value of each entry on the diagonal is greater than the sum of the absolute values of the other entries in the same row, i.e if $|a_{00}|>|a_{01}+a_{02}+\cdots|$. In general it can be shown that a iterative scheme $\mathbf{x}^{k+1}=\mathbf{P}\cdot \mathbf{x}^k+\mathbf{q}$ is convergent *if and only if* every eigenvalue, $\lambda$, of $\mathbf{P}$ satisfies $|\lambda|<1$, i.e. the *spectral radius* $\rho(\mathbf{P})<1$.



## The Gauss-Seidel method
It is tempting in equation ([50](#eq:lin:jc3a)) to use our estimate of $x_0^{k+1}$ in the next equation, equation ([51](#eq:lin:jc3b)), instead of $x_0^k$. After all our estimate $x_0^{k+1}$ is an *improved* estimate. This is actually the Gauss-Seidel method. This method also has the advantage that if there are memory issues, one can overwrite the old value of $x_i^k$. Usually the Gauss-Seidel method converges faster, but not always. A plus for the Jacobi method is that is can be  parallelised, as the calculations is only dependent on the old values and do not require information about the new values as for the Gauss Seidel method. Below is a Python implementation of the Gauss-Seidel method

In [9]:
def solve_GS(A,b,x=-1,max_iter=1000,EPS=1e-6):
    """
    Solves the linear system Ax=b using the Gauss-Seidel method, stops if
    solution is not found after max_iter or if solution changes less 
    than EPS
    """
    if(x==-1):
        x=np.zeros(len(b))
    D=np.diag(A)
    R=A-np.diag(D)
    eps=1
    iter=0
    while(eps>EPS and iter<max_iter):
        iter+=1
        eps=0.
        for i in range(len(x)):
            tmp=x[i]
            x[i]=(b[i]- np.dot(R[i,:],x))/D[i]
            eps+=np.abs(tmp-x[i])
    print('found solution after ' + str(iter) +' iterations')
    return x

Run the code below to test the Gauss-Seidel method

In [10]:
A = np.array([[2, 1, 1, 3,],[1, 4, 1, 1, ],
              [1, 1, 3, 1],[1, 1, 2, 2 ]],float)
b = np.array([1,2,-3,1],float)

s3b=solve_GS(A,b)
print(s3b)

# Example: Linear regression
In the previous section, we considered a system of $N$ equations and $N$ unknown ($x_0, x_1,\ldots, x_N$). In general we might have more equations than unknowns or more unknowns than equations. An example of the former is linear regression, we might have many data points and we would like to fit a line through the points. How do you fit a single lines to more than two points that does not line on the same line? One way to do it is to minimize the distance from the line to the points, as illustrated in [figure 4](#fig:lin:reg).
<!-- dom:FIGURE: [fig-lin/reg.png, width=800 frac=.9] Linear regression by minimizing the total distance to all the points. <div id="fig:lin:reg"></div> -->
<!-- begin figure -->
<div id="fig:lin:reg"></div>

<img src="fig-lin/reg.png" width=800><p style="font-size: 0.9em"><i>Figure 4: Linear regression by minimizing the total distance to all the points.</i></p>
<!-- end figure -->

Mathematically we can express the distance between a data point $(x_i,y_i)$ and the line $f(x)$ as $y_i-f(x_i)$. Note that this difference can be negative or positive depending if the data point lies below or above the line. We can then take the absolute value of all the distances, and try to minimize them. When we minimize something we take the derivative of the expression and put it equal to zero.  As you might remember from Calculus it is extremely hard to work with the derivative of the absolute value, because it is discontinuous. A much better approach is to square each distance and sum them:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:lsq"></div>

$$
\begin{equation}
S=\sum_{i=0}^{N-1}(y_i-f(x_i))^2=\sum_{i=0}^{N-1}(y_i-a_0-a_1x_i)^2.
\label{eq:lin:lsq} \tag{54}
\end{equation}
$$

(For the example in [figure 4](#fig:lin:reg), $N=5$.) This is the idea behind *least square*, and linear regression. One thing you should be aware of is that points lying far from the line will contribute more to equation ([54](#eq:lin:lsq)). The underlying assumption is that each data point provides equally precise information about the process, this is often not the case. When analyzing experimental data, there may be points deviating from the expected behaviour, it is then important to investigate if these points are more affected by measurements errors than the others. If that is the case one should give them less weight in the least square estimate, by extending the formula above:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:lsqm"></div>

$$
\begin{equation}
S=\sum_{i=0}^{N-1}\omega_i(y_i-f(x_i))^2=\sum_{i=0}^3\omega_i(y_i-a_0-a_1x_i)^2,
\label{eq:lin:lsqm} \tag{55}
\end{equation}
$$

$\omega_i$ is a weight factor.

## Solving least square, using algebraic equations
Let us continue with equation ([54](#eq:lin:lsq)), the algebraic solution is to simply find the value of $a_0$ and $a_1$ that minimizes $S$:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:ls1"></div>

$$
\begin{equation}
\frac{\partial S}{\partial a_0} =-2\sum_{i=0}^{N-1}(y_i-a_0-a_1x_i)=0,
\label{eq:lin:ls1} \tag{56} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:ls2"></div>

$$
\begin{equation}  
\frac{\partial S}{\partial a_1} =-2\sum_{i=0}^{N-1}(y_i-a_0-a_1x_i)x_i=0.
\label{eq:lin:ls2} \tag{57}
\end{equation}
$$

Defining the mean value as $\overline{x}=\sum_ix_i/N$ and $\overline{y}=\sum_iy_i/N$, we can write equation ([56](#eq:lin:ls1)) and ([57](#eq:lin:ls2))  as:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:ls1a"></div>

$$
\begin{equation}
\sum_{i=0}^{N-1}(y_i-a_0-a_1x_i)=N\overline{y}-a_0N-a_1N\overline{x}=0,
\label{eq:lin:ls1a} \tag{58} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:ls2b"></div>

$$
\begin{equation}  
\sum_{i=0}^{N-1}(y_i-a_0-a_1x_i)x_i=\sum_iy_ix_i-a_0N\overline{x}-a_1\sum_ix_ix_i=0.
\label{eq:lin:ls2b} \tag{59}
\end{equation}
$$

Solving equation ([58](#eq:lin:ls1a)) with respect to $a_0$, and inserting the expression into equation ([59](#eq:lin:ls2b)), we find:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:ls1c"></div>

$$
\begin{equation}
a_0=\overline{y}-a_1\overline{x},\label{eq:lin:ls1c} \tag{60} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:ls2d"></div>

$$
\begin{equation}  
a_1=\frac{\sum_iy_ix_i-N\overline{x}\overline{y}}{\sum_ix_i^2-N\overline{x}^2}
=\frac{\sum_i(x_i-\overline{x})(y_i-\overline{y})}{\sum_i(x_i-\overline{x})^2}.
\label{eq:lin:ls2d} \tag{61}
\end{equation}
$$

We leave it as an exercise to show the last expression for $a_1$.  
Clearly the equation ([61](#eq:lin:ls2d)) above will in most cases have
a solution. But in addition to a solution, it would be good to have an
idea of the goodness of the fit. Intuitively it make sense to add all
the distances (residuals) $d_i$ in [figure 4](#fig:lin:reg). This is
basically what is done when calculating $R^2$ (R-squared). However, we
would also like to compare the $R^2$ between different
datasets. Therefor we need to normalize the sum of residuals, and
therefore the following form of the $R^2$ is used:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:r2"></div>

$$
\begin{equation}
R^2=1-\frac{\sum_{i=0}^{N-1}(y_i-f(x_i))^2}{\sum_{i=0}^{N-1}(y_i-\overline{y})^2}.
\label{eq:lin:r2} \tag{62}
\end{equation}
$$

In python we can implement equation ([60](#eq:lin:ls1c)), ([61](#eq:lin:ls2d)) and ([62](#eq:lin:r2)) as:

In [11]:
def OLS(x, y): 
    # returns regression coefficients
    # in ordinary least square
    # x: observations
    # y: response
    # R^2: R-squared
    n = np.size(x) # number of data points 
  
    # mean of x and y vector 
    m_x, m_y = np.mean(x), np.mean(y) 
  
    # calculating cross-deviation and deviation about x 
    SS_xy = np.sum(y*x) - n*m_y*m_x 
    SS_xx = np.sum(x*x) - n*m_x*m_x 
  
    # calculating regression coefficients 
    b_1 = SS_xy / SS_xx 
    b_0 = m_y - b_1*m_x

    #R^2
    y_pred = b_0 + b_1*x
    S_yy   = np.sum(y*y) - n*m_y*m_y
    y_res  = y-y_pred  
    S_res  = np.sum(y_res*y_res)
  
    return(b_0, b_1,1-S_res/S_yy)

## Least square as a linear algebra problem
It turns out that the least square problem can be formulated as a
matrix problem. (Two great explanations see [linear regression by
matrices](https://medium.com/@andrew.chamberlain/the-linear-algebra-view-of-least-squares-regression-f67044b7f39b),
and
[$R^2$-squared](https://medium.com/@andrew.chamberlain/a-more-elegant-view-of-r-squared-a0a14c177dc3).)
If we define a matrix $\mathbf{X}$ containing the observations $x_i$
as:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:mreg1"></div>

$$
\begin{equation}
\mathbf{X} =
\begin{pmatrix}
1&x_0\\ 
1&x_1\\ 
\vdots&\vdots\\ 
1&x_{N-1}
\end{pmatrix}.
\label{eq:lin:mreg1} \tag{63}
\end{equation}
$$

We introduce a vector containing all the response $\mathbf{y}$, and the
regression coefficients $\mathbf{a}=(a_0,a_1)$. Then we can write
equation ([55](#eq:lin:lsqm)) as a matrix equation:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:mregS"></div>

$$
\begin{equation}
S=(\mathbf{y}-\mathbf{X\cdot a})^T(\mathbf{y}-\mathbf{X\cdot a}).
\label{eq:lin:mregS} \tag{64}
\end{equation}
$$

*Note that this equation can easily be extended to more than one
observation variable $x_i$*. By simply differentiating equation
([64](#eq:lin:mregS)) with respect to $\mathbf{a}$, we can show that
the derivative has a minimum when (see proof below):

<!-- Equation labels as ordinary links -->
<div id="eq:lin:mregS2"></div>

$$
\begin{equation}
\mathbf{X}^T\mathbf{X a}=\mathbf{X}^T\mathbf{y}
\label{eq:lin:mregS2} \tag{65}
\end{equation}
$$

Below is a python implementation of equation ([65](#eq:lin:mregS2)).

In [12]:
def OLSM(x, y): 
    # returns regression coefficients
    # in ordinary least square using solve function
    # x: observations
    # y: response

    XT = np.array([np.ones(len(x)),x],float)
    X  = np.transpose(XT)
    B = np.dot(XT,X)
    C = np.dot(XT,y)
    return solve(B,C)

## Working with matrices on component form
Whenever you want to do some manipulation with matrices, it is very useful to simply write them on component form. If we multiply two matrices $\mathbf{A}$ and $\mathbf{B}$ to form a new matrix $\mathbf{C}$, the components of the new matrix is simply $\mathbf{C}_{ij}=\sum_k\mathbf{A}_{ik}\mathbf{B}_{kj}$. The strength of doing this is that the elements of a matrix, e.g. $\mathbf{A}_{ik}$ are *numbers*, and we can move them around. Proving that e.g. $(\mathbf{A}\mathbf{B})^T=\mathbf{B}^T\mathbf{A}^T$ is straight forward using the component form. The transpose of a matrix is simply to exchange columns and rows, hence $\mathbf{C}_{ij}^T=\mathbf{C}_{ji}$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:trans"></div>

$$
\begin{equation}
\mathbf{C}_{ij}^T=\mathbf{C}_{ji}=\sum_k\mathbf{A}_{jk}\mathbf{B}_{ki}=\sum_k\mathbf{B}^T_{ik}\mathbf{A}^T_{kj}
=(\mathbf{B}^T\mathbf{A}^T)_{ij},
\label{eq:lin:trans} \tag{66}
\end{equation}
$$

thus $\mathbf{C}^T=\mathbf{B}^T\mathbf{A}^T$. To derive equation ([65](#eq:lin:mregS2)), we need to take the derivative of equation ([65](#eq:lin:mregS2)) with respect to $\mathbf{a}$.
What we mean by this is that we want to evaluate $\partial S/\partial a_k$ for all the components of $\mathbf{a}$.
A useful rule is $\partial a_i/\partial a_k=\delta_{ik}$, where $\delta_{ik}$ is the Kronecker delta, it takes the value of one if $i=k$ and zero otherwise. We can write $S=\mathbf{y}^T\mathbf{y}-\mathbf{y}\mathbf{X\cdot a}
-(\mathbf{X\cdot a})^T\mathbf{y}-(\mathbf{X\cdot a})^T\mathbf{X\cdot a}$. All terms that do not contain $\mathbf{a}$ are zero, thus we only need to evaluate the following terms

<!-- Equation labels as ordinary links -->
<div id="_auto12"></div>

$$
\begin{equation}
\frac{\partial}{a_k}(\mathbf{X\cdot a})^T\mathbf{y} =\frac{\partial}{a_k}(\mathbf{a}^T\cdot \mathbf{X}^T\mathbf{y})=\frac{\partial}{a_k}\sum_{ij}\mathbf{a}^T_i\mathbf{X}^T_{ij}\mathbf{y}_j
=\sum_{ij}\delta_{ik}\mathbf{X}^T_{ij}\mathbf{y}_j{\nonumber}
\label{_auto12} \tag{67}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto13"></div>

$$
\begin{equation}  
=\sum_{j}\mathbf{X}^T_{kj}\mathbf{y}_j=\mathbf{X}^T\mathbf{y} 
\label{_auto13} \tag{68}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto14"></div>

$$
\begin{equation}  
\frac{\partial}{a_k}\mathbf{y}^T\mathbf{X\cdot a}=\frac{\partial}{a_k}\sum_{ij}\mathbf{y}^T_i\mathbf{X}_{ij}\mathbf{a}_j
=\sum_{ij}\mathbf{y}^T_i\mathbf{X}_{ij}\delta_{jk}=\sum_{j}\mathbf{y}^T_{i}\mathbf{X}_{ik}{\nonumber}
\label{_auto14} \tag{69}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto15"></div>

$$
\begin{equation}  
=\sum_{j}\mathbf{y}^T_{i}\mathbf{X}^T_{ki}=\mathbf{X}^T\mathbf{y} 
\label{_auto15} \tag{70}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto16"></div>

$$
\begin{equation}  
\frac{\partial}{a_k} (\mathbf{X\cdot a})^T\mathbf{X\cdot a}=
\frac{\partial}{a_k}\sum_{ijl} \mathbf{a}^T_i\mathbf{X}^T_{ij}\mathbf{X}_{jl}\mathbf{a}_l=
\sum_{ijl}(\delta_{ik}\mathbf{X}^T_{ij}\mathbf{X}_{jl}\mathbf{a}_l+\mathbf{a}^T_i\mathbf{X}^T_{ij}\mathbf{X}_{jl}\delta_{lk}){\nonumber}
\label{_auto16} \tag{71}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto17"></div>

$$
\begin{equation} =\sum_{jl}\mathbf{X}^T_{kj}\mathbf{X}_{jl}
\mathbf{a}_l+\sum_{ij}\mathbf{a}^T_i\mathbf{X}^T_{ij}\mathbf{X}_{jk}{\nonumber}
\label{_auto17} \tag{72}
\end{equation}
$$

$$
\begin{equation}  
=\mathbf{X}^T\mathbf{X}\mathbf{a}+\sum_{ij}\mathbf{X}^T_{kj}\mathbf{X}_{ji}\mathbf{a}_i
= 2\mathbf{X}^T\mathbf{X}\mathbf{a}.
\label{}
\end{equation}
$$

It then follows that $\partial S/\partial \mathbf{a} = 0$ when

<!-- Equation labels as ordinary links -->
<div id="eq:lin:matpr"></div>

$$
\begin{equation}
\mathbf{X}^T\mathbf{X a}=\mathbf{X}^T\mathbf{y}.
\label{eq:lin:matpr} \tag{73}
\end{equation}
$$

# Sparse matrices and Thomas algorithm
In many practical examples, such as solving partial differential
equations the matrices could be quite large and also contain a lot of
zeros. A very important class of such matrices are *banded matrices*
this is a type of *sparse matrices* containing a lot of zero elements,
and the non-zero elements are confined to diagonal bands. In the
following we will focus on one important type of sparse matrix the
tridiagonal. In the next section we will show how it enters naturally
in solving the heat equation. It turns out that solving banded
matrices is quite simple, and can be coded quite efficiently. As with
the Gauss-Jordan example, lets consider a concrete example:

<!-- Equation labels as ordinary links -->
<div id="_auto18"></div>

$$
\begin{equation}
\left(
\begin{array}{ccccc|c}
b_0&c_0&0&0&0&r_0\\ 
a_1&b_1&c_1&0&0&r_1\\ 
0&a_2&b_2&c_2&0&r_2\\ 
0& 0&a_3&b_3&c_3&r_3\\ 
0& 0& 0&a_4&b_4&r_4
\end{array}
\right)
\label{_auto18} \tag{74}
\end{equation}
$$

The right hand side is represented with $r_i$. The first Gauss-Jordan
step is simply to divide by $b_0$, then we multiply with $-a_1$ and
add to second row:

<!-- Equation labels as ordinary links -->
<div id="_auto19"></div>

$$
\begin{equation}
\to \left(
\begin{array}{ccccc|c}
1&c_0^\prime&0&0&0&r_0^\prime\\ 
0&b_1-a_1c_0^\prime&c_1&0&0&r_1-a_0r_0^\prime\\ 
0&a_2&b_2&c_2&0&r_2\\ 
0& 0&a_3&b_3&c_3&r_3\\ 
0& 0& 0&a_4&b_4&r_4
\end{array}
\right),
\label{_auto19} \tag{75}
\end{equation}
$$

Note that we have introduced some new symbols to simplify the
notation: $c_0^\prime=c_0/b_0$ and $r_0^\prime=r_0/b_0$. Then we
divide by $b_1-a_1c_0^\prime$:

<!-- Equation labels as ordinary links -->
<div id="_auto20"></div>

$$
\begin{equation}
\left(
\begin{array}{ccccc|c}
1&c_0^\prime&0&0&0&r_0^\prime\\ 
0&1&c_1^\prime&0&0&r_1^\prime\\ 
0&a_2&b_2&c_2&0&r_2\\ 
0& 0&a_3&b_3&c_3&r_3\\ 
0& 0& 0&a_4&b_4&r_4
\end{array}
\right),
\label{_auto20} \tag{76}
\end{equation}
$$

where $c_1^\prime=c_1/(b_1-a_1c_0^\prime)$ and
$r_1^\prime=(r_1-a_0r_0^\prime)/(b_1-a_1c_0^\prime)$. If you continue
in this manner, you can easily convince yourself that to transform a
tridiagonal matrix to the following form:

<!-- Equation labels as ordinary links -->
<div id="_auto21"></div>

$$
\begin{equation}
\to \left(
\begin{array}{ccccc|c}
1&c_0^\prime&0&0&0&r_0^\prime\\ 
0&1&c_1^\prime&0&0&r_1^\prime\\ 
0&0&1&c_2^\prime&0&r_2^\prime\\ 
0& 0&0&1&c_3^\prime&r_3^\prime\\ 
0& 0& 0&0&1&r_4^\prime
\end{array}
\right),
\label{_auto21} \tag{77}
\end{equation}
$$

where:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:th0"></div>

$$
\begin{equation}
c_0^\prime =\frac{c_0}{b_0} \qquad r_0^\prime=\frac{r_0}{b_0}
\label{eq:lin:th0} \tag{78} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:thi"></div>

$$
\begin{equation}  
c_i^\prime
=\frac{c_i}{b_i-a_ic_{i-1}^\prime}\qquad
r_i^\prime=\frac{r_i-a_ir_{i-1}^\prime}{b_i-a_ic_{i-1}^\prime}
\quad\text{, for }i=1,2,\ldots,N-1\label{eq:lin:thi} \tag{79} 
\end{equation}
$$

Note that we where able to reduce the tridiagonal matrix to an *upper
triangular* matrix in only *one* Gauss-Jordan step. This equation can
readily be solved using back-substitution, which can also be
simplified as there are a lot of zeros in the upper part. Let us
denote the unknowns $x_i$ as we did for the Gauss-Jordan case, now we
can find the solution as follows:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:this0"></div>

$$
\begin{equation}
x_{N-1}  = r_{N-1}^\prime \label{eq:lin:this0} \tag{80} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:thisi"></div>

$$
\begin{equation}  
x_i      = r_i^\prime-x_{i+1}c_i^\prime\quad\text{, for } i=N-2,N-3,\ldots,0
\label{eq:lin:thisi} \tag{81}
\end{equation}
$$

Equation ([78](#eq:lin:th0)), ([79](#eq:lin:thi)), ([80](#eq:lin:this0))
and ([81](#eq:lin:thisi)) is known as the Thomas algorithm after
Llewellyn Thomas. 
**Notice.**

Clearly tridiagonal matrices can be solved much more efficiently with
the Thomas algorithm than
using a standard library, such as LU-decomposition. This is
because the solution method takes advantages of the *symmetry* of the
problem. We will not show it here, but it can be shown that the Thomas
algorithm is stable whenever $|b_i|\ge |a_i|+|c_i|$. If the algorithm
fails, an advice is first to use the standard `solve` function in
python. If this gives a solution, then *pivoting* combined with the
Thomas algorithm might do the trick.


# Example: Solving the heat equation using linear algebra

<!-- --- begin exercise --- -->

## Exercise 1: Conservation Equation or the Continuity Equation

<!-- dom:FIGURE: [fig-lin/heat.png, width=700 frac=.9] Conservation of energy and the continuity equation. <div id="fig:nlin:heat"></div> -->
<!-- begin figure -->
<div id="fig:nlin:heat"></div>

<img src="fig-lin/heat.png" width=700><p style="font-size: 0.9em"><i>Figure 5: Conservation of energy and the continuity equation.</i></p>
<!-- end figure -->

In [figure 5](#fig:nlin:heat), the continuity equation is derived for
heat flow.
### Heat equation for solids

As derived in the beginning of this chapter the heat equation for a solid is

<!-- Equation labels as ordinary links -->
<div id="eq:nlin:heateq"></div>

$$
\begin{equation}
\frac{d^2T}{dx^2}+\frac{\dot{\sigma}}{k}=\frac{\rho c_p}{k}\frac{dT}{dt},
\label{eq:nlin:heateq} \tag{82}
\end{equation}
$$

where $\dot{\sigma}$ is the rate of heat generation in the solid. This
equation can be used as a starting point for many interesting
models. In this exercise we will investigate the *steady state*
solution, *steady state* is just a fancy way of expressing that we
want the solution that *does not change with time*. This is achieved
by ignoring the derivative with respect to time in equation
([82](#eq:nlin:heateq)). We want to study a system with size $L$, and is
it good practice to introduce a dimensionless variable: $y=x/L$. 

**Part 1.**

Show that equation ([82](#eq:nlin:heateq)) now takes the following form:

<!-- Equation labels as ordinary links -->
<div id="eq:nlin:heat2"></div>

$$
\begin{equation}
\frac{d^2T }{dy^2}+\frac{\dot{\sigma}L^2}{k}=0
\label{eq:nlin:heat2} \tag{83}
\end{equation}
$$

<!-- --- end exercise --- -->

<!-- --- begin exercise --- -->

## Exercise 2: Curing of Concrete and Matrix Formulation

Curing of concrete is one particular example that we can investigate
with equation ([83](#eq:nlin:heat2)). When concrete is curing, there are
a lot of chemical reactions happening, these reactions generate
heat. This is a known issue, and if the temperature rises too much 
compared to the surroundings, the concrete may fracture.  In the
following we will, for simplicity, assume that the rate of heat
generated during curing is constant, $\dot{\sigma}=$100 W/m$^3$. The
left end (at $x=0$) is insulated, meaning that there is no flow of
heat over that boundary, hence $dT/dx=0$ at $x=0$. On the right hand
side the temperature is kept constant, $x(L)=y(1)=T_1$, assumed to be
equal to the ambient temperature of $T_1=25^\circ$C.  The concrete
thermal conductivity is assumed to be $k=1.65$ W/m$^\circ$C.

**Part 1.**

Show that the solution to equation ([83](#eq:nlin:heat2)) in this case is:

<!-- Equation labels as ordinary links -->
<div id="eq:nlin:heatsol"></div>

$$
\begin{equation}
T(y)=\frac{\dot{\sigma}L^2}{2k}(1-y^2)+T_1.
\label{eq:nlin:heatsol} \tag{84}
\end{equation}
$$

**Part 2.**
In order to solve equation ([83](#eq:nlin:heat2)) numerically, we need to discretize
it. Show that equation ([83](#eq:nlin:heat2)) now takes the following form:

<!-- Equation labels as ordinary links -->
<div id="eq:nlin:heat3"></div>

$$
\begin{equation}
T_{i+1}+T_{i-1}-2T_i=-h^2\beta,
\label{eq:nlin:heat3} \tag{85}
\end{equation}
$$

where $\beta=\dot{\sigma}L^2/k$.
<!-- dom:FIGURE: [fig-lin/heat_grid.png, width=200 frac=.5] Finite difference grid for $N=4$. <div id="fig:nlin:hgrid"></div>  -->
<!-- begin figure -->
<div id="fig:nlin:hgrid"></div>

<img src="fig-lin/heat_grid.png" width=200><p style="font-size: 0.9em"><i>Figure 6: Finite difference grid for $N=4$.</i></p>
<!-- end figure -->

In [figure 6](#fig:nlin:hgrid), the finite difference grid is shown for
$N=4$.

**Part 3.**

Show that equation ([85](#eq:nlin:heat3)) including the boundary conditions for $N=4$ can be written as the following matrix equation

<!-- Equation labels as ordinary links -->
<div id="eq:lin:heats"></div>

$$
\begin{equation}
\left(
\begin{array}{cccc}
-\gamma&\gamma&0&0\\ 
1&-2&1&0\\ 
0&1&-2&1\\ 
0&0&1&-2\\ 
\end{array}
\right)
\left(
\begin{array}{c}
T_0\\ 
T_1\\ 
T_2\\ 
T_3\\ 
\end{array}
\right)
=
\left(
\begin{array}{c}
-h^2\beta\\ 
-h^2\beta\\ 
-h^2\beta\\ 
-h^2\beta-25
\end{array}
\right).
\end{equation}
\label{eq:lin:heats} \tag{86}
$$

where $\gamma=2$ for the central difference scheme and 1 for the forward difference scheme.

**Part 4.**
* Solve the set of equations in equation ([86](#eq:lin:heats)) using [`numpy.linalg.solve`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html).

* Write the code so that you can easily switch between the central difference scheme and forward difference

* Evaluate the numerical error as you change $h$, how does it scale? Is it what you expect?

In [13]:
import numpy as np
import scipy as sc
import scipy.sparse.linalg
from numpy.linalg import solve
import matplotlib.pyplot as plt

In [14]:

central_difference=False
# set simulation parameters
h=0.25
L=1.0
n = int(round(L/h))
Tb=25 #rhs
sigma=100
k=1.65 
beta = sigma*L**2/k

y = np.arange(n+1)*h

def analytical(x):
    return beta*(1-x*x)/2+Tb
def tri_diag(a, b, c, k1=-1, k2=0, k3=1):
    """ a,b,c diagonal terms
        default k-values for 4x4 matrix:
        | b0 c0 0  0 |
        | a0 b1 c1 0 |
        | 0  a1 b2 c2|
        | 0  0  a2 b3|
    """
    return np.diag(a, k1) + np.diag(b, k2) + np.diag(c, k3)
# defina a, b and c vector
a=np.ones(n-1)
b=..
c=..

if central_difference:
    c[0]= ...
else:
    b[0]=...

A=tri_diag(a,b,c)
print(A) # view matrix - compare with N=4 to make sure no bugs
# define rhs vector
d=...
#rhs boundary condition
d[-1]=...

Tn=np.linalg.solve(A,d)
print(Tn)

The correct solution for $L=1$ m, and $h=1/4$, is: $[T_0,T_1.T_2,T_3]$=[55.3030303 , 53.40909091, 47.72727273, 38.25757576] (central difference) and $[T_0,T_1.T_2,T_3]$=[62.87878788, 59.09090909, 51.51515152, 40.15151515] (forward difference)

<!-- --- end exercise --- -->

<!-- --- begin exercise --- -->

## Exercise 3: Solve the full heat equation

**Part 1.**
Replace the time derivative in equation ([82](#eq:nlin:heateq)) with

<!-- Equation labels as ordinary links -->
<div id="eq:lin:dt"></div>

$$
\begin{equation}
\frac{dT}{dt}\simeq\frac{T(t+\Delta t)-T(t)}{\Delta t}=\frac{T^{n+1}-T^n}{\Delta t}, 
\label{eq:lin:dt} \tag{87}
\end{equation}
$$

and show that by using an *implicit formulation* (i.e. that the second derivative with respect to $x$ is to be evaluated at $T(t+\Delta t)\equiv T^{n+1}$) that equation ([82](#eq:nlin:heateq)) can be written

<!-- Equation labels as ordinary links -->
<div id="eq:lin:imp"></div>

$$
\begin{equation}
T_{i+1}^{n+1}+T_{i-1}^{n+1}-(2+\frac{\alpha h^2}{\Delta t})T_i^{n+1}=-h^2\beta-\frac{\alpha h^2 }{\Delta t}T_i^n,
\label{eq:lin:imp} \tag{88} 
\end{equation}
$$

where $\alpha\equiv\rho c_p/k$.

**Part 2.**

Use the central difference formulation for the boundary condition and show that for four nodes we can formulate equation ([88](#eq:lin:imp)) as the following matrix equation

<!-- Equation labels as ordinary links -->
<div id="_auto22"></div>

$$
\begin{equation}
\left(
\begin{array}{cccc}
-(2+\frac{\alpha h^2}{\Delta t})&2&0&0\\ 
1&-(2+\frac{\alpha h^2}{\Delta t})&1&0\\ 
0&1&-(2+\frac{\alpha h^2}{\Delta t})&1\\ 
0&0&1&-(2+\frac{\alpha h^2}{\Delta t})\\ 
\end{array}
\right)
\left(
\begin{array}{c}
T_0^{n+1}\\ 
T_1^{n+1}\\ 
T_2^{n+1}\\ 
T_3^{n+1}\\ 
\end{array}
\right){\nonumber}
\label{_auto22} \tag{89}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:heatfull"></div>

$$
\begin{equation}  
=
\left(
\begin{array}{c}
-h^2\beta\\ 
-h^2\beta\\ 
-h^2\beta\\ 
-h^2\beta-25
\end{array}
\right)
-\frac{\alpha h^2 }{\Delta t}
\left(
\begin{array}{c}
T_0^n\\ 
T_1^n\\ 
T_2^n\\ 
T_3^n\\ 
\end{array}
\right)
\end{equation}
\label{eq:lin:heatfull} \tag{90}
$$

**Part 3.**
Assume that the initial temperature in the concrete is $25^\circ$C, $\rho$=2400 kg/m$^3$, a specific heat capacity $c_p=$ 1000 W/kg K, and a time step of $\Delta t=86400$ s (1 day). Solve equation ([90](#eq:lin:heatfull)), plot the result each day and compare the result after 50 days with the steady state solution in equation ([84](#eq:nlin:heatsol)).

<!-- --- end exercise --- -->

<!-- --- begin exercise --- -->

## Exercise 4: Using sparse matrices in python

In this part we are going to create a sparse matrix in python and use `scipy.sparse.linalg.spsolve` to solve it. The matrix is created using `scipy.sparse.spdiags`.

**Part 1.**
Extend the code you developed in the last exercises to also be able to use sparse matrices, by e.g. a logical switch. Sparse matrices may be defined as follows

In [15]:
import scipy.sparse.linalg

#right hand side
# rhs vector
d=np.repeat(-h*h*beta,n)
#rhs - constant temperature
Tb=25
d[-1]=d[-1]-Tb
#Set up sparse matrix
diagonals=np.zeros((3,n))
diagonals[0,:]= 1
diagonals[1,:]= -2  
diagonals[2,:]= 1
#No flux boundary condition
diagonals[2,1]= 2
A_sparse = sc.sparse.spdiags(diagonals, [-1,0,1], n, n,format='csc')
# to view matrix - do this and check that it is correct!
print(A_sparse.todense())
# solve matrix
Tb = sc.sparse.linalg.spsolve(A_sparse,d)

# if you like you can use timeit to check the efficiency
# %timeit sc.sparse.linalg.spsolve( ... )

* Compare the sparse solver with the standard Numpy solver using `%timeit`, how large must the linear system be before an improvement in speed is seen?

<!-- --- end exercise --- -->

# CO$_2$ diffusion into aquifers
The transport of CO$_2$ into aquifers can be described according to the diffusion equation

<!-- Equation labels as ordinary links -->
<div id="eq:lin:co2_diff"></div>

$$
\begin{equation}
\frac{\partial C(z,t)}{\partial t}=\frac{\partial }{\partial z}\left(K(z)\frac{\partial C(z,t)}{\partial z}\right),
\label{eq:lin:co2_diff} \tag{91}
\end{equation}
$$

where $C(z,t)$ is the concentration of {CO$_2$}\ as a function of depth ($z$) and time $t$, and $K(z)$ is the diffusion constant of {CO$_2$}\ as a function of depth. This equation can be discretized using standard techniques, to help in that respect consider [figure 7](#fig:lin:co2_diff).

<!-- dom:FIGURE: [fig-lin/co2_diff.png, width=800 frac=1.0] Discretization for diffusion of {CO$_2$}\ into an aquifer, including boundary conditions. <div id="fig:lin:co2_diff"></div> -->
<!-- begin figure -->
<div id="fig:lin:co2_diff"></div>

<img src="fig-lin/co2_diff.png" width=800><p style="font-size: 0.9em"><i>Figure 7: Discretization for diffusion of {CO$_2$}\ into an aquifer, including boundary conditions.</i></p>
<!-- end figure -->

In the following we will assume that there are only four nodes ($i=0\ldots 3$) in the physical domain, and two ghost nodes $i=-1$, and $i=4$. There are many ways to attack this problem, but in the following we will borrow ideas from Finite Volume. Finite volume methods is a way of discretizing equations such that we *conserve mass*. The diffusion equation as it is derived in [figure 5](#fig:nlin:heat), express that the flux of something (heat, particles, etc) leaving the box surface minus the flux entering the surface of the box is equal to the rate of change of something inside the box. We can formulate this mathematically as:

<!-- Equation labels as ordinary links -->
<div id="eq:lin:co2fv"></div>

$$
\begin{equation}
\frac{\partial C(z,t)}{\partial t}\simeq\frac{1}{h}\left[\left.K(z)\frac{\partial C(z,t)}{\partial z}\right|_{i+1/2}
-\left.K(z)\frac{\partial C(z,t)}{\partial z}\right|_{i-1/2}\right]
\label{eq:lin:co2fv} \tag{92}
\end{equation}
$$

The notation $i\pm1/2$, means that the flux is to be evaluated *at the surface* of the box (i.e. halfway between the red dots in [figure 7](#fig:lin:co2_diff)). $K(z)$ is the diffusion constant, and it is known everywhere, so this is simple to evaluate at the surface. The concentrations are only known at the center of each box, the red dots in [figure 7](#fig:lin:co2_diff). The derivative of the concentration can be evaluated using the central difference formula (remember that the distance between the red dot and edge of the box is $h/2$), hence

<!-- Equation labels as ordinary links -->
<div id="eq:lin:co2fv2"></div>

$$
\begin{equation}
\frac{C_i^{n+1}-C_i^n}{\Delta t}=\frac{1}{h}\left[K_{i+1/2}\frac{ C_{i+1}-C_{i}}{h}-K_{i-1/2}\frac{ C_{i}-C_{i-1}}{h}\right],
\label{eq:lin:co2fv2} \tag{93}
\end{equation}
$$

notice that we have discretized the time derivative, and that we have introduced $n$ to indicate the time step. On the right hand side there are is no time indicated, it turns out that we have a choice to put time step $n$ or $n+1$ on the concentrations on the right hand side. If we put $n$ the scheme is said to be explicit, if we put $n+1$, the scheme is implicit. Implicit schemes are stable compared to explicit schemes, whereas explicit schemes has slightly higher numerical accuracy [TO DO 1: show this!]. In general we can write

<!-- Equation labels as ordinary links -->
<div id="_auto23"></div>

$$
\begin{equation}
\frac{C_i^{n+1}-C_i^n}{\Delta t}=\frac{\theta}{h}\left[K_{i+1/2}
\frac{C^n_{i+1}-C^n_{i}}{h}-K_{i-1/2}\frac{C^n_{i}-C^n_{i-1}}{h}\right]
\label{_auto23} \tag{94}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:co2fv23"></div>

$$
\begin{equation}  
+\frac{1-\theta}{h}\left[K_{i+1/2}
\frac{C_{i+1}^{n+1}-C_{i}^{n+1}}{h}-K_{i-1/2}\frac{ C_{i}^{n+1}-C_{i-1}^{n+1}}{h}\right],
\label{eq:lin:co2fv23} \tag{95}
\end{equation}
$$

hence if $\theta=1$ the scheme is explicit, if $\theta=0$ the scheme is implicit, and if $\theta=1/2$, the scheme is called the Crank-Nicolson method. The first and last boundary are special, let us first consider the $i=0$, this is where the sea is in contact with the {CO$_2$}\ in the atmosphere, and the flux is $k_w(C_0-C_{eq})$, hence

<!-- Equation labels as ordinary links -->
<div id="_auto24"></div>

$$
\begin{equation}
\frac{C_0^{n+1}-C_0^n}{\Delta t}=\frac{\theta}{h}\left[K_{i+1/2}
\frac{C^n_{1}-C^n_{0}}{h}-k_w(C_0^n-C_{eq}^n)\right]
\label{_auto24} \tag{96}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:co2fv23b"></div>

$$
\begin{equation}  
+\frac{1-\theta}{h}\left[K_{i+1/2}
\frac{C_{1}^{n+1}-C_{0}^{n+1}}{h}-k_w(C_0^{n+1}-C_{eq}^{n+1})\right].
\label{eq:lin:co2fv23b} \tag{97}
\end{equation}
$$

For the last block the flux is zero towards the seafloor, and equation ([95](#eq:lin:co2fv23)) can be written

<!-- Equation labels as ordinary links -->
<div id="_auto25"></div>

$$
\begin{equation}
\frac{C_3^{n+1}-C_3^n}{\Delta t}  =\frac{\theta}{h}\left[-K_{5/2}\frac{C^n_{3}-C^n_{2}}{h}\right]
\label{_auto25} \tag{98}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:co2fv23c"></div>

$$
\begin{equation}  
+\frac{1-\theta}{h}\left[-K_{5/2}\frac{ C_{3}^{n+1}-C_{2}^{n+1}}{h}\right].
\label{eq:lin:co2fv23c} \tag{99}
\end{equation}
$$

For the blocks $i=1\ldots2$, we can collect all terms with $n+1$ on one side and terms with $n$ on the other side and rewrite equation ([95](#eq:lin:co2fv23))

<!-- Equation labels as ordinary links -->
<div id="_auto26"></div>

$$
\begin{equation}
\left[1+(1-\theta)\right. \left.\alpha(K_{i+1/2}+K_{i-1/2})\right]C_i^{n+1}{\nonumber}
\label{_auto26} \tag{100}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto27"></div>

$$
\begin{equation}  
-(1-\theta)\alpha K_{i+1/2}C_{i+1}^{n+1}
-(1-\theta)\alpha K_{i-1/2}C_{i-1}^{n+1} {\nonumber}
\label{_auto27} \tag{101}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto28"></div>

$$
\begin{equation}  
=\left[1-\theta\right. \left.\alpha(K_{i+1/2}+K_{i-1/2})\right]C_i^{n}{\nonumber}
\label{_auto28} \tag{102}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:co2fv4"></div>

$$
\begin{equation}  
+\theta\alpha K_{i+1/2}C_{i+1}^{n}
+\theta\alpha K_{i-1/2}C_{i-1}^{n}, 
\label{eq:lin:co2fv4} \tag{103}
\end{equation}
$$

where $\alpha\equiv\Delta t/h^2$.
Next, we want to write down the corresponding matrix equations for four grid nodes as indicated in [figure 7](#fig:lin:co2_diff). Notice that we need to use the equations in [figure 7](#fig:lin:co2_diff), for $C_{-1}$, and $C_4$. The left and right hand coefficient matrix $\mathbf{L}$, and $\mathbf{R}$ are given as

1
1
3
 
<
<
<
!
!
M
A
T
H
_
B
L
O
C
K

<!-- Equation labels as ordinary links -->
<div id="_auto30"></div>

$$
{\tiny
\begin{equation}
\begin{pmatrix}
1-\theta\alpha(K_{1/2}+hk_w)&+\theta\alpha K_{1/2}&0&0\\ 
\theta\alpha K_{1/2}&1-\theta\alpha(K_{3/2}+K_{1/2})&\theta\alpha K_{3/2} &0\\ 
0&\theta\alpha K_{3/2}&1-\theta\alpha(K_{5/2}+K_{3/2})&\theta\alpha K_{5/2} \\ 
0&0&\theta\alpha K_{5/2}&1-\theta\alpha K_{5/2}{\nonumber}
\end{pmatrix},
\label{_auto30} \tag{105}
\end{equation}
}
$$

respectively. Introducing $\mathbf{S}=\left[k_wC_{eq}\Delta t/h,0,0,0\right]^T$, we can finally write the diffusion equation ([91](#eq:lin:co2_diff)) as

<!-- Equation labels as ordinary links -->
<div id="eq:lin:discdif"></div>

$$
\begin{equation}
\mathbf{L}\mathbf{C}^{n+1}=\mathbf{R}\mathbf{C}^n+\theta\mathbf{S}^n+(1-\theta)\mathbf{S}^{n+1}
\label{eq:lin:discdif} \tag{106}
\end{equation}
$$

More stuff to do:
1. Assume zero flux over the air water interface ($k_w$=0), show from the equations above that if we start with a uniform concentration in the sea ($\mathbf{C}^n$=constant) that $\mathbf{C}^{n+1}$ does not change (as it should).

2. Assume that if the concentration at a specific time $n$ in the sea is equal to $\mathbf{C}_{eq}$ then the concentration stays constant at all later times

3. Add chemical reactions

# References

1. <div id="leveque2002finite"></div> **R. J. LeVeque et al.**.  *Finite Volume Methods for Hyperbolic Problems*, Cambridge University Press, 2002.

2. <div id="press2007"></div> **W. H. Press, S. A. Teukolsky, W. T. Vetterling and B. P. Flannery**.  *Numerical Recipes 3rd Edition: the Art of Scientific Computing*, 3 edition, Cambridge University Press, 2007.

3. <div id="trefethen1997"></div> **L. N. Trefethen and D. B. III**.  *Numerical Linear Algebra*, SIAM, 1997.

4. <div id="stoer2013"></div> **J. Stoer and R. Bulirsch**.  *Introduction to Numerical Analysis*, Springer Science & Business Media, 2013.

5. <div id="strang2019"></div> **G. Strang**.  *Linear Algebra and Learning From Data*, Wellesley-Cambridge Press, 2019.

6. <div id="press2001"></div> **W. H. Press, W. T. Vetterling, S. A. Teukolsky and B. P. Flannery**.  *Numerical Recipes in C++: the Art of Scientific Computing*, 2nd edition, Cambridge University Press, 2002.